In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv ('CO2V2.Temperature.csv')
#df = pd.read_csv ('CO2V2.Humidity.csv')
#print(df)

In [3]:
df.head()

Seconds   Latitude  Longitude  Elevation         Data
0  2022-09-22T09:30:19.708989Z  51.539353  -0.147692    92266.6  2492.118812
1  2022-09-22T09:30:20.708989Z  51.539354  -0.147690    92052.6  2494.000000
2  2022-09-22T09:30:21.708989Z  51.539353  -0.147689    91940.9  2495.920000
3  2022-09-22T09:30:22.708989Z  51.539354  -0.147690    91888.0  2497.000000
4  2022-09-22T09:30:23.708989Z  51.539354  -0.147690    91958.9  2498.930693

In [4]:
len(df)

2226

In [5]:
df=df.dropna()

In [6]:
len(df)

2206

In [7]:
# Generate Locations
#for index, row in df.iterrows():
 #   print('{{"name": "location", "description": "GPS coordinates of a data point.","encodingType": "application/vnd.geo+json","location": {{"type": "Point","coordinates": [{} ,{}],"time": "{} "}}}},'.format(df['Longitude'][index],df['Latitude'][index],df['Seconds'][index]))

In [8]:
#Generate Data
#for index, row in df.iterrows():
#    print('{{"phenomenonTime": "{}","result": {}}},'.format(df['Seconds'][index],df['Data'][index]))

In [9]:
#!pip install frost-sta-client

In [10]:
import frost_sta_client as fsc

In [11]:
url = "http://localhost/FROST-Server/v1.1"
#url = "https://emotional.byteroad.net/FROST-Server/v1.1"

In [12]:
PASSWORD = os.environ.get('FROST_PASSWORD')
#print(PASSWORD)

In [13]:
auth_handler=fsc.AuthHandler(username="admin",password=PASSWORD )

In [14]:
service = fsc.SensorThingsService(url,auth_handler)

In [15]:
# Use this to delete stuff
#things_list = service.things().query().list()

#for thing in things_list:
#    service.delete(thing)

In [16]:
things_list = service.things().query().list()
for thing in things_list:
    print("my name is: {}".format(thing.name))

In [17]:
# create thing
thing = fsc.Thing(name='Backpack',
              description='Backpack equiped with sensors', properties={'withLocation': True})

In [18]:
from geojson import Point
from geojson import dumps, loads

In [19]:
# create locations
locations=[]
for index, row in df.iterrows():
    point = Point((df['Longitude'][index], df['Latitude'][index]))
    dump = dumps(point, sort_keys=True)
    load=loads(dump)
    location = fsc.Location(name="location", description="GPS coordinates of a data point", location=load,
                            encoding_type='application/geo+json', properties={'time': df['Seconds'][index]})
    locations.append(location)
    #print (location.location)


In [20]:
#point = Point((df['Longitude'][0], df['Latitude'][0]))
#print(point)
#dump = dumps(point, sort_keys=True)
#load=loads(dump)

#location = fsc.Location(name="location", description="GPS coordinates of a data point", location=point,
 #                           encoding_type='application/geo+json')

In [21]:
#point = Point((-0.147692, 51.539353))
#print(load)
#location = fsc.Location(name="here", description="and there", location=load, encoding_type='application/geo+json')
#thing = fsc.Thing(name='new thing',
#              description='I am a thing with a location',
#              properties={'withLocation': True, 'owner': 'IOSB'})

In [22]:
thing.locations = locations

In [23]:
try:
    service.create(thing)
except Exception as e:
    print(e)

In [25]:
#Create observedProperty
observedProperty=fsc.ObservedProperty(
                 name='Temperature',
                 definition='http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#ThermodynamicTemperature',
                 description='The temperature.')

In [26]:
#Create sensor
sensor=fsc.Sensor(
				name="DHT22/Temperature",
				description="Temperature sensor of a DHT22",
				encoding_type="application/pdf",
				metadata="https://www.sparkfun.com/datasheets/Sensors/Temperature/DHT22.pdf")

In [27]:
import frost_sta_client.model.ext.unitofmeasurement

unit= frost_sta_client.model.ext.unitofmeasurement.UnitOfMeasurement(
                 name="Centigrade",
                 symbol="C",
                 definition="http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html#DegreeCentigrade")


In [28]:
datastream = fsc.Datastream(name='CO2 Temperature',
              description='The temperature in CO2', 
                            observation_type="http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
                           unit_of_measurement=unit,
                            observed_property=observedProperty, sensor=sensor, thing=thing)

In [29]:
service.create(datastream)

In [30]:
 
    dav = fsc.model.ext.data_array_value.DataArrayValue()

    components = {dav.Property.PHENOMENON_TIME, dav.Property.RESULT, dav.Property.FEATURE_OF_INTEREST}
    #components = {dav.Property.PHENOMENON_TIME, dav.Property.RESULT}
    dav.components = components
    dav.datastream = datastream
        
    for index, row in df.iterrows():
            dump = dumps(df['Data'][index], sort_keys=True)
            load=loads(dump)
        
            point = Point((df['Longitude'][index], df['Latitude'][index]))
            dump2 = dumps(point, sort_keys=True)
            load2=loads(dump2)
            
            
            foi = fsc.FeatureOfInterest(name="gps location", 
                                        description="GPS coordinates of a data point.", 
                                        encoding_type="application/vnd.geo+json",
                                       feature=load2)
            service.create(foi)
            #print(foi.feature)
        
            obs = fsc.Observation(result= load,
                           phenomenon_time=df['Seconds'][index],
                           datastream=datastream, feature_of_interest=foi)
            #print(obs)
            #print(df['Data'][index],df['Seconds'][index])
            dav.add_observation(obs)
    
    dad = fsc.model.ext.data_array_document.DataArrayDocument()
    dad.add_data_array_value(dav)
    result_list = service.observations().create(dad)

In [31]:
# repeat for humidity
df = pd.read_csv ('CO2V2.Humidity.csv')

In [32]:
df=df.dropna()

In [33]:
len(df)

2206

In [34]:
#Create observedProperty
observedProperty=fsc.ObservedProperty(
                 name='Relative Humidity',
                 definition='https://en.wikipedia.org/wiki/Relative_humidity',
                 description='The relative humidity')

In [35]:
#Create sensor
sensor=fsc.Sensor(
				name="DHT22/Humidity",
				description="Relative humidity sensor of a DHT22",
				encoding_type="application/pdf",
				metadata="https://www.sparkfun.com/datasheets/Sensors/Temperature/DHT22.pdf")

In [36]:
import frost_sta_client.model.ext.unitofmeasurement

unit= frost_sta_client.model.ext.unitofmeasurement.UnitOfMeasurement(
                 name="percentage",
                 symbol="%",
                 definition="hhttps://en.wikipedia.org/wiki/Percentage")

In [37]:
datastream = fsc.Datastream(name='CO2 Humidity',
              description='The Humidity in CO2', 
                            observation_type="http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
                           unit_of_measurement=unit,
                            observed_property=observedProperty, sensor=sensor, thing=thing)

In [38]:
service.create(datastream)

In [39]:
 
    dav = fsc.model.ext.data_array_value.DataArrayValue()

    components = {dav.Property.PHENOMENON_TIME, dav.Property.RESULT}
    dav.components = components
    dav.datastream = datastream
        
    for index, row in df.iterrows():
            dump = dumps(df['Data'][index], sort_keys=True)
            load=loads(dump)
        

            point = Point((df['Longitude'][index], df['Latitude'][index]))
            dump2 = dumps(point, sort_keys=True)
            load2=loads(dump2)
            
            
            foi = fsc.FeatureOfInterest(name="gps location", 
                                        description="GPS coordinates of a data point.", 
                                        encoding_type="application/vnd.geo+json",
                                       feature=load2)
            service.create(foi)      
        
            obs = fsc.Observation(result= load,
                           phenomenon_time=df['Seconds'][index],
                           datastream=datastream, feature_of_interest=foi)
            #print(obs)
            #print(df['Data'][index],df['Seconds'][index])
            dav.add_observation(obs)
    
    dad = fsc.model.ext.data_array_document.DataArrayDocument()
    dad.add_data_array_value(dav)
    result_list = service.observations().create(dad)